In [2]:
from imageai.Detection import ObjectDetection

Using TensorFlow backend.


In [3]:
import os

In [4]:
execution_path = os.getcwd()

In [5]:
detector = ObjectDetection()

In [6]:
detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "mother_fuckin_image.jpg"), output_image_path=os.path.join(execution_path , "imagenew1.jpg"))

Instructions for updating:
Colocations handled automatically by placer.
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


In [14]:
for eachObject in detections:
    print(eachObject)
    print([int((eachObject['box_points'][2:][0] + eachObject['box_points'][:2][0])/2), int((eachObject['box_points'][2:][1] + eachObject['box_points'][:2][1])/2)])

{'name': 'car', 'percentage_probability': 52.85045504570007, 'box_points': [156, 9, 174, 27]}
[165, 18]
{'name': 'person', 'percentage_probability': 60.029393434524536, 'box_points': [88, 77, 97, 105]}
[92, 91]
{'name': 'person', 'percentage_probability': 58.618372678756714, 'box_points': [75, 80, 86, 108]}
[80, 94]
{'name': 'person', 'percentage_probability': 51.33609175682068, 'box_points': [63, 84, 74, 113]}
[68, 98]
{'name': 'car', 'percentage_probability': 52.01278328895569, 'box_points': [140, 35, 165, 55]}
[152, 45]
{'name': 'car', 'percentage_probability': 81.60916566848755, 'box_points': [222, 36, 245, 56]}
[233, 46]
{'name': 'car', 'percentage_probability': 60.60898303985596, 'box_points': [141, 52, 171, 90]}
[156, 71]
{'name': 'car', 'percentage_probability': 64.5590603351593, 'box_points': [94, 116, 150, 171]}
[122, 143]
{'name': 'truck', 'percentage_probability': 69.25075054168701, 'box_points': [31, 136, 132, 237]}
[81, 186]
{'name': 'bus', 'percentage_probability': 59.40

In [9]:
import numpy as np

quad_coords = {
    "lonlat": np.array([
        [51.538444, -0.102410], # corner of back telephone box
        [51.538615, -0.102251], # far bus stop marking
        [51.538229, -0.102508], # Post in front left
        [51.538191, -0.102353] # Corner of dashed line bottom right
    ]),
    "pixel": np.array([
        [131, 80],
        [213, 32],
        [25, 217],
        [342, 281]
    ])
}

In [10]:
import cv2


class PixelMapper(object):
    """
    Create an object for converting pixels to geographic coordinates,
    using four points with known locations which form a quadrilteral in both planes
    Parameters
    ----------
    pixel_array : (4,2) shape numpy array
        The (x,y) pixel coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    lonlat_array : (4,2) shape numpy array
        The (lon, lat) coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    """
    def __init__(self, pixel_array, lonlat_array):
        assert pixel_array.shape==(4,2), "Need (4,2) input array"
        assert lonlat_array.shape==(4,2), "Need (4,2) input array"
        self.M = cv2.getPerspectiveTransform(np.float32(pixel_array),np.float32(lonlat_array))
        self.invM = cv2.getPerspectiveTransform(np.float32(lonlat_array),np.float32(pixel_array))
        
    def pixel_to_lonlat(self, pixel):
        """
        Convert a set of pixel coordinates to lon-lat coordinates
        Parameters
        ----------
        pixel : (N,2) numpy array or (x,y) tuple
            The (x,y) pixel coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (lon, lat) coordinates
        """
        if type(pixel) != np.ndarray:
            pixel = np.array(pixel).reshape(1,2)
        assert pixel.shape[1]==2, "Need (N,2) input array" 
        pixel = np.concatenate([pixel, np.ones((pixel.shape[0],1))], axis=1)
        lonlat = np.dot(self.M,pixel.T)
        
        return (lonlat[:2,:]/lonlat[2,:]).T
    
    def lonlat_to_pixel(self, lonlat):
        """
        Convert a set of lon-lat coordinates to pixel coordinates
        Parameters
        ----------
        lonlat : (N,2) numpy array or (x,y) tuple
            The (lon,lat) coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (x, y) pixel coordinates
        """
        if type(lonlat) != np.ndarray:
            lonlat = np.array(lonlat).reshape(1,2)
        assert lonlat.shape[1]==2, "Need (N,2) input array" 
        lonlat = np.concatenate([lonlat, np.ones((lonlat.shape[0],1))], axis=1)
        pixel = np.dot(self.invM,lonlat.T)
        
        return (pixel[:2,:]/pixel[2,:]).T


In [11]:
pm = PixelMapper(quad_coords["pixel"], quad_coords["lonlat"])

In [12]:
uv_0 = (131, 80) # Top left give way sign in frame
lonlat_0 = pm.pixel_to_lonlat(uv_0)
uv_2 = (213, 32)
lonlat_2 = pm.pixel_to_lonlat(uv_2)
uv_3 = (100,120)
lonlat_3 = pm.pixel_to_lonlat(uv_3)
uv_4 = (159,195)
lonlat_4 = pm.pixel_to_lonlat(uv_4)
uv_5 = (300, 110)
lonlat_5 = pm.pixel_to_lonlat(uv_5)

lonlat_1 = (6.603361, 52.036639) # Center of the roundabout on googlemaps
uv_1 = pm.lonlat_to_pixel(lonlat_1)

In [13]:
lonlat_4

array([[51.53824717, -0.10241868]])

In [ ]:
51.538348, -0.102444